In [ ]:
import pandas as pd
from Codigos.BaseDeDados.cleaner import Cleaner, Corretor
from tqdm.auto import tqdm

In [ ]:
objetos_limpo = pd.read_csv('/var/projetos/Jupyterhubstorage/victor.silva/HelBERTModel/Datasets/Objetos/df_tipos_objetos_lowercase_sem_tokenizacao.csv')
objetos_limpo

In [ ]:
objetos_raw_original = pd.read_csv('/var/projetos/Jupyterhubstorage/victor.silva/HelBERTModel/Datasets/Objetos/DatasetsRaw/DFObjetos.csv')
objetos_raw_original

In [ ]:
objetos_raw_original.dropna(subset=['objeto'], inplace=True)
objetos_raw_original.drop_duplicates(subset=['objeto'], inplace=True)
objetos_raw_original.reset_index(drop=True, inplace=True)
objetos_raw_original

In [ ]:
objetos_raw = objetos_raw_original.copy()

In [ ]:
def executaLimpeza(dataframe: pd.DataFrame, column: str, cased: bool, accents: bool) -> pd.DataFrame:
    """
    Executa a limpeza do DataFrame usando as classes Cleaner e Corretor.
    """
    sentencesDrop = []
    for indice in tqdm(dataframe.index, desc="Executando a Limpeza", colour='yellow'):
        texto = dataframe[column][indice]
        if isinstance(texto, str):
            texto = Cleaner().clear(texto)
            texto = Corretor(cased, accents).corrige_termos(texto)
            dataframe.at[indice, column] = texto
        else:
            sentencesDrop.append(indice)
    dataframe = dataframe.drop(sentencesDrop)
    return dataframe

In [ ]:
objetos_raw = executaLimpeza(objetos_raw, 'objeto', cased=False, accents=False)
objetos_raw

In [ ]:
lista_indices_raw = []
for indice_raw in tqdm(objetos_raw.index, desc="Comparando os DataFrames", colour='yellow'):
    objeto_raw = objetos_raw['objeto'][indice_raw]
    for indice_limpo in objetos_limpo.index:
        objeto_limpo = objetos_limpo['text'][indice_limpo]
        if objeto_raw.find(objeto_limpo) != -1:
            lista_indices_raw.append([objetos_raw_original['objeto'][indice_raw], objetos_limpo['label'][indice_limpo]])
            break

In [ ]:
lista_indices_raw

In [ ]:
dados = pd.DataFrame(lista_indices_raw, columns=['text', 'label'])
dados

In [ ]:
dados_raw_achados = pd.read_csv("/var/projetos/Jupyterhubstorage/victor.silva/HelBERTModel/Datasets/Objetos/DatasetsRaw/df_objetos_tipos_raw.csv")
dados_raw_achados

In [ ]:
dados_raw_rotulados = pd.read_csv("/var/projetos/Jupyterhubstorage/victor.silva/HelBERTModel/Datasets/Objetos/DatasetsRaw/df_tipos_objetos_raw_completo.csv")
dados_raw_rotulados

In [ ]:
dados = pd.concat([dados, dados_raw_achados, dados_raw_rotulados], ignore_index=True)
dados

In [ ]:
dados.to_csv('/var/projetos/Jupyterhubstorage/victor.silva/HelBERTModel/Datasets/Objetos/DatasetsRaw/df_tipos_objetos_raw_completo.csv', index=False)

### Criando a base de objetos sintética, substituindo <numero> por números aleatórios

In [ ]:
import random
import re
import pandas as pd

In [ ]:
dados = pd.read_csv("/var/projetos/Jupyterhubstorage/victor.silva/HelBERTModel/Datasets/Objetos/df_tipos_objetos_cased.csv")
dados

In [ ]:
def generate_random_number(match):
    return str(random.randint(1, 100))

In [ ]:
for indice in dados.index:
    dados.loc[indice, 'text'] = re.sub('(<EMAIL>)', 'email', dados.loc[indice, 'text'], flags=re.IGNORECASE)
    dados.loc[indice, 'text'] = re.sub('(<DATA>)', generate_random_number, dados.loc[indice, 'text'], flags=re.IGNORECASE)
    dados.loc[indice, 'text'] = re.sub('(<HORA>)', generate_random_number, dados.loc[indice, 'text'], flags=re.IGNORECASE)
    dados.loc[indice, 'text'] = re.sub('(<TELEFONE>)', generate_random_number, dados.loc[indice, 'text'], flags=re.IGNORECASE)
    dados.loc[indice, 'text'] = re.sub('(<CEP>)', 'cep', dados.loc[indice, 'text'], flags=re.IGNORECASE)
    dados.loc[indice, 'text'] = re.sub('(<URL>)', 'url', dados.loc[indice, 'text'], flags=re.IGNORECASE)
    dados.loc[indice, 'text'] = re.sub('((<NUMERO>)|(number))', generate_random_number, dados.loc[indice, 'text'], flags=re.IGNORECASE)

In [ ]:
dados

In [ ]:
dados.to_csv('/var/projetos/Jupyterhubstorage/victor.silva/HelBERTModel/Datasets/Objetos/df_tipos_objetos_cased_com_num_sintetico.csv', index=False)

In [ ]:
import pandas as pd

In [ ]:
dados = pd.read_csv("/var/projetos/Jupyterhubstorage/victor.silva/HelBERTModel/Datasets/Objetos/DatasetsRaw/df_tipos_objetos_raw_completo.csv")
dados

In [ ]:
nao_duplicados = dados.drop_duplicates(subset=['text'], keep='first')
nao_duplicados

In [ ]:
duplicados = dados[~dados.index.isin(nao_duplicados.index)]
duplicados

In [ ]:
regex_adicionar_ponto = r'(?<!\.)$'

# Adicionando ponto final
texto_com_ponto = re.sub(regex_adicionar_ponto, '.', texto)

In [ ]:
regex_remover_ponto = r'\.$'

# Removendo ponto final
texto_sem_ponto = re.sub(regex_remover_ponto, '', texto_com_ponto)

In [ ]:
for indice in duplicados.index:
    duplicados.loc[indice, 'text'] = re.sub(regex_adicionar_ponto, '.', duplicados.loc[indice, 'text'])

In [ ]:
for indice in nao_duplicados.index:
    nao_duplicados.loc[indice, 'text'] = re.sub(regex_remover_ponto, '', nao_duplicados.loc[indice, 'text'])

In [ ]:
dados = pd.concat([nao_duplicados, duplicados], ignore_index=True)
dados

In [ ]:
dados.dropna(subset=['text'], inplace=True)
dados.drop_duplicates(subset=['text'], inplace=True)
dados.reset_index(drop=True, inplace=True)
dados

In [ ]:
dados.to_csv('/var/projetos/Jupyterhubstorage/victor.silva/HelBERTModel/Datasets/Objetos/DatasetsRaw/df_tipos_objetos_raw_completo.csv', index=False)

In [ ]:
def executaLimpeza(dataframe: pd.DataFrame, column: str, cased: bool, accents: bool) -> pd.DataFrame:
    """
    Executa a limpeza do DataFrame usando as classes Cleaner e Corretor.
    """
    sentencesDrop = []
    for indice in tqdm(dataframe.index, desc="Executando a Limpeza", colour='yellow'):
        texto = dataframe[column][indice]
        if isinstance(texto, str):
            texto = Cleaner().clear(texto)
            texto = Corretor(cased, accents).corrige_termos(texto)
            dataframe.at[indice, column] = texto
        else:
            sentencesDrop.append(indice)
    dataframe = dataframe.drop(sentencesDrop)
    return dataframe

In [ ]:
dados = executaLimpeza(dados, 'text', cased=True, accents=True)
dados

In [ ]:
dados.dropna(subset=['text'], inplace=True)
dados.drop_duplicates(subset=['text'], inplace=True)
dados.reset_index(drop=True, inplace=True)
dados

In [ ]:
dados['label'].value_counts()

In [ ]:
dados.to_csv('/var/projetos/Jupyterhubstorage/victor.silva/HelBERTModel/Datasets/Objetos/DatasetsGold/df_tipos_objetos_cased.csv', index=False)

In [ ]:
dados_objetos = pd.read_csv("/var/projetos/Jupyterhubstorage/victor.silva/HelBERTModel/Datasets/Objetos/DatasetsGold/df_tipos_objetos_cased.csv")
dados_objetos

In [ ]:
dados_objetos['label'].value_counts()